In [2]:
import os
import time
import glob
import shutil
import fnmatch
import pathlib

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [5]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder

"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome-debug"

In [9]:
import numpy as np
import pandas as pd

In [12]:
import datetime as dt

In [14]:
from IPython.display import display

In [16]:
from openpyxl import load_workbook

In [18]:
import nsepython

In [20]:
indexes = ['nifty 50', 'nifty 100', 'nifty 200', 'nifty 500', 'nifty largemidcap 250', 'nifty midcap 50', 'nifty midcap 100', 'nifty midcap 150', 'nifty smallcap 50', 'nifty smallcap 100', 'nifty smallcap 250']

In [22]:
start_date = "25-Sep-2024"
end_date = "29-Sep-2024"

In [24]:
nsepython.index_total_returns(indexes[4], start_date, end_date)

,RequestNumber,Index Name,Date,TotalReturnsIndex
0,TRI63863201130455112000,NIFTY LargeMidcap 250,27 Sep 2024,21895.39
1,TRI63863201130455112000,NIFTY LargeMidcap 250,26 Sep 2024,21899.76
2,TRI63863201130455112000,NIFTY LargeMidcap 250,25 Sep 2024,21811.64


In [26]:
nsepython.index_pe_pb_div(indexes[4], start_date, end_date)

""


# Save a new Excel file

Uncomment below code to download and save data to new excel file.

In [30]:
# tri = nsepython.index_total_returns('nifty largemidcap 250', "01-Mar-2011", "01-Mar-2024")
# tri

In [32]:
# pe = nsepython.index_pe_pb_div('nifty largemidcap 250', "01-Mar-2011", "01-Mar-2024")
# pe

In [34]:
# df = pd.merge(tri.iloc[:, 2:], pe.iloc[:, 2:], left_on='Date', right_on='DATE', how='inner')[:, :-1]
# df.to_excel('nifty largemidcap 250.xlsx')

# Get the data since the last saved date

In [37]:
def get_data(index):
    '''get TRI history and PE history from nsepython as dataframe'''
    
    #last date in the file
    last = pd.read_excel(index+'.xlsx', index_col=0).index[-1]

    # get values of the remaining dates
    tri = nsepython.index_total_returns(index, (last.date()+dt.timedelta(days=1)).strftime("%d-%b-%Y"),
                                        dt.date.today().strftime("%d-%b-%Y"))
    pe = nsepython.index_pe_pb_div(index, (last.date()+dt.timedelta(days=1)).strftime("%d-%b-%Y"),
                               dt.date.today().strftime("%d-%b-%Y"))
    tri_pe = pd.merge(tri.iloc[:, 2:], pe.iloc[:, 2:], 
                      left_on='Date', right_on='DATE', how='inner').iloc[:, :-1]
    tri_pe[tri_pe.columns[:1]] = tri_pe.iloc[:, :1].apply(pd.to_datetime)
    tri_pe[tri_pe.columns[1:]] = tri_pe.iloc[:, 1:].apply(pd.to_numeric)
    tri_pe.sort_values(by='Date', ascending=True, inplace=True)
    return tri_pe

In [39]:
def save_data(index):
    '''save the index data to existing excel files'''

    wb = load_workbook(index+'.xlsx')
    ws = wb.active
    
    try:
        tri_pe = get_data(index)
    except KeyError:
        print("error for " + index)
        return

    for row in tri_pe.itertuples(index=False):
        ws.append([row.Date, row.TotalReturnsIndex, row.pe, row.pb, row.divYield])
    wb.save(index+'.xlsx')
    
    print('saved for '+ index)

In [41]:
for i in indexes[:]:
    save_data(i)

saved for nifty 50
saved for nifty 100
saved for nifty 200
saved for nifty 500
error for nifty largemidcap 250
saved for nifty midcap 50
saved for nifty midcap 100
saved for nifty midcap 150
saved for nifty smallcap 50
error for nifty smallcap 100
saved for nifty smallcap 250


display last two daya of data from excel file

# Scrape and save the data

In [45]:
error_indices = ['nifty largemidcap 250', 'nifty smallcap 100']

In [47]:
chrome_options = Options()
chrome_options.debugger_address = "127.0.0.1:9222"
driver = webdriver.Chrome(options=chrome_options)

driver.implicitly_wait(3)
driver.get("https://www.niftyindices.com/reports/historical-data")

In [49]:
def scrape_data(index):
    data_element = driver.find_element(By.ID, "HistoricalMenu")
    time.sleep(3)
    data_element.click()
    time.sleep(3)
    
    data_dropdown = driver.find_element(By.CSS_SELECTOR, ".form4")
    data_dropdown.click()
    
    index_type = Select(driver.find_element(By.ID, "ddlHistoricaldivtypee"))
    index_type.select_by_visible_text("Equity")
    
    index_select = driver.find_element(By.ID, "ddlHistoricaldivtypeeindex")
    Select(index_select).select_by_visible_text(index.upper())
    
    date_picker = driver.find_element(By.ID, "datepickerFromDivYield")
    date_picker.click()
    
    month = driver.find_element(By.CLASS_NAME, "ui-datepicker-month")
    Select(month).select_by_value("8")
    
    date = driver.find_element(By.XPATH, "/html/body/div/table/tbody/tr[2]/td[1]")
    date.click()
    
    submit = driver.find_element(By.ID, "submit_buttonDivdata")
    time.sleep(1)
    submit.click()

    download = driver.find_element(By.ID, "exporthistoricaldiv")
    # driver.execute_script('arguments[0].scrollIntoView(true)', download)
    ActionChains(driver).scroll_by_amount(0, -500).perform()
    time.sleep(1)
    download.click()

    time.sleep(2)
    basepath = 'C:/Users/dsanr/Downloads/'
    for file in os.listdir(basepath):
        if fnmatch.fnmatch(file, '*csv'):
            print(file)
            data_file = pathlib.Path(os.path.join(basepath, file))
            os.rename(data_file, os.path.join(basepath, 'data.csv'))
            shutil.copy(os.path.join(basepath, 'data.csv'), "D:/projects/mf")
            os.remove(os.path.join(basepath, 'data.csv'))

In [51]:
def merge_data(index):
    '''get TRI history and PE history from nsepython as dataframe'''
    
    #last date in the file
    last = pd.read_excel(index+'.xlsx', index_col=0).index[-1]

    # get values of the remaining dates
    tri = nsepython.index_total_returns(index, (last.date()+dt.timedelta(days=1)).strftime("%d-%b-%Y"),
                                        dt.date.today().strftime("%d-%b-%Y"))
    scrape_data(index)
    pe = pd.read_csv('data.csv')
    
    tri_pe = pd.merge(tri.iloc[:, 2:], pe.iloc[:, 1:], left_on='Date', right_on='Date')
    
    tri_pe[tri_pe.columns[:1]] = tri_pe.iloc[:, :1].apply(pd.to_datetime)
    tri_pe[tri_pe.columns[1:]] = tri_pe.iloc[:, 1:].apply(pd.to_numeric)
    tri_pe.sort_values(by='Date', ascending=True, inplace=True)
    return tri_pe

In [53]:
def save_data(index):
    '''save the index data to existing excel files'''

    wb = load_workbook(index+'.xlsx')
    ws = wb.active
    
    tri_pe = merge_data(index)

    for row in tri_pe.itertuples(index=False):
        ws.append([row.Date, row.TotalReturnsIndex, row._2, row._3, row._4])
    wb.save(index+'.xlsx')
    
    print('saved for '+ index)

In [55]:
for i in error_indices:
    save_data(i)

null_Historical_PE_PB_DIV_Data_08092024to29092024.csv
saved for nifty largemidcap 250
null_Historical_PE_PB_DIV_Data_08092024to29092024.csv
saved for nifty smallcap 100


# Check the data

In [58]:
for i in indexes:
    print(i)
    display(pd.read_excel(i+'.xlsx').tail(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
3404,2024-09-26,38916.76,24.38,3.87,1.16
3405,2024-09-27,38861.64,24.34,3.87,1.16


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
3404,2024-09-26,36983.63,25.35,4.03,1.14
3405,2024-09-27,36980.38,25.35,4.03,1.14


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
3405,2024-09-26,11463.46,27.57,3.85,0.82
3406,2024-09-27,11438.17,27.51,3.84,0.83


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
3404,2024-09-26,24711.00,29.99,4.07,0.8
3405,2024-09-27,24686.28,29.96,4.07,0.8


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
3405,2024-09-26,23287.12,31.82,4.26,0.84
3406,2024-09-27,23242.76,31.76,4.25,0.84


To check whether same date repeats

In [317]:
for i in indexes:
    df=pd.read_excel(i+'.xlsx', index_col=0)
    print(i)
    display(df[df.index.duplicated(keep=False)])
    print(f"total duplicated indexes {df.index.duplicated(keep=False).sum()}\n")

nifty 50


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 200


...

total duplicated indexes 2

nifty smallcap 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty smallcap 250


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,
2016-04-01,4742.59,65.64,1.54,1.14
2016-04-01,4742.59,86.86,1.35,1.20


total duplicated indexes 2



Stating date for which P/E values are available

In [319]:
for i in indexes:
    print(i)
    df = pd.read_excel(i+'.xlsx')
    display(df[~df['P/E'].isnull()].head(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,7727.31,24.57,3.88,1.01
1,2011-01-04,7713.22,24.53,3.87,1.01


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,6924.52,23.15,3.76,1.01
1,2011-01-04,6908.94,23.10,3.75,1.02


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,3089.9,24.93,1.20,1.29
1302,2016-04-01,3089.9,27.61,0.99,1.52


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
1204,2015-11-09,6227.22,22.55,1.02,1.45
1205,2015-11-10,6144.27,22.5,1.00,1.47


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,4742.59,65.64,1.54,1.14
1302,2016-04-01,4742.59,86.86,1.35,1.20


Examples

In [71]:
sbi = nsepython.equity_history('sbin', series='EQ', start_date="01-09-2024", end_date="09-09-2024")
sbi

,_id,CH_SYMBOL,CH_SERIES,CH_MARKET_TYPE,CH_TIMESTAMP,TIMESTAMP,CH_TRADE_HIGH_PRICE,CH_TRADE_LOW_PRICE,CH_OPENING_PRICE,CH_CLOSING_PRICE,...,CH_52WEEK_HIGH_PRICE,CH_52WEEK_LOW_PRICE,CH_TOTAL_TRADES,CH_ISIN,createdAt,updatedAt,__v,SLBMH_TOT_VAL,VWAP,mTIMESTAMP
0,66d6777bb88d538f051f8377,SBIN,EQ,N,2024-09-02,2024-09-01T18:30:00.000Z,825.40,813.05,817.30,822.15,...,912,543.2,279222,INE062A01020,2024-09-03T02:42:03.345Z,2024-09-03T02:42:03.345Z,0,None,819.67,02-Sep-2024
1,66d71a0062903c2b898b9c47,SBIN,EQ,N,2024-09-03,2024-09-02T18:30:00.000Z,825.95,816.40,822.15,824.80,...,912,543.2,197793,INE062A01020,2024-09-03T14:15:28.309Z,2024-09-03T14:15:28.309Z,0,None,821.71,03-Sep-2024
2,66d84f16e28bd85dd9bafde8,SBIN,EQ,N,2024-09-04,2024-09-03T18:30:00.000Z,818.65,813.65,815.45,816.50,...,912,543.2,177040,INE062A01020,2024-09-04T12:14:14.341Z,2024-09-04T12:14:14.341Z,0,None,816.16,04-Sep-2024
3,66d99d605db4cad1ee61fd64,SBIN,EQ,N,2024-09-05,2024-09-04T18:30:00.000Z,822.15,814.40,818.35,818.75,...,912,543.2,146739,INE062A01020,2024-09-05T12:00:32.549Z,2024-09-05T12:00:32.549Z,0,None,819.29,05-Sep-2024
4,66db1216216ed5a318698f41,SBIN,EQ,N,2024-09-06,2024-09-05T18:30:00.000Z,812.90,777.00,811.50,782.50,...,912,543.2,753751,INE062A01020,2024-09-06T14:30:46.247Z,2024-09-06T14:30:46.247Z,0,None,790.89,06-Sep-2024


# Old



nsepy package with Total Returns Index history is available at https://github.com/dsanr/nsepy

In [5]:
import nsepy

In [6]:
indexes = ['nifty 50', 'nifty 100', 'nifty 200', 'nifty 500', 'nifty largemidcap 250', 'nifty midcap 50', 'nifty midcap 100', 'nifty midcap 150',
           'nifty smallcap 50', 'nifty smallcap 100', 'nifty smallcap 250']

In [7]:
dt.date.today()

datetime.date(2023, 3, 8)

In [8]:
nsepy.get_total_returns_index_history('nifty 500', dt.date.today()-dt.timedelta(days=3), dt.date.today())

,Total Returns Index
Date,
2023-03-06,23108.97
2023-03-08,23172.99


## Downloading and saving data to new excel file

Uncomment below code to download and save data to new excel file.

In [9]:
# tri = nsepy.get_total_returns_index_history('nifty largemidcap 250', dt.date(2021,1,1), dt.date(2020,12,22))
# tri

In [10]:
# pe = nsepy.get_index_pe_history('nifty largemidcap 250', dt.date(2021,1,1), dt.date(2020,12,22))
# pe

In [11]:
# df = tri.join(pe)
# df.to_excel('nifty largemidcap 250.xlsx')

## Downloading the data for the days since the last saved date

In [12]:
def get_data(index):
    '''get TRI history and PE history from nsepy as dataframe'''
    
    #last date in the file
    last = pd.read_excel(index+'.xlsx', index_col=0).index[-1]
    
    # get values of the remaining dates
    tri = nsepy.get_total_returns_index_history(index, last.date()+dt.timedelta(days=1), dt.date.today())
    pe = nsepy.get_index_pe_history(index, last.date()+dt.timedelta(days=1), dt.date.today())
    tri_pe = tri.join(pe)
    return tri_pe

Saving downloaded data to existing excel file

In [13]:
# def save_data(index):
#     '''save the index data to existing excel files'''
    
#     tri_pe = get_data(index)
    
#     existing_data = pd.read_excel(index+'.xlsx', index_col=0)
#     with pd.ExcelWriter(index+'.xlsx', date_format='YYYY-MM-DD') as writer:
#         pd.concat([existing_data, tri_pe]).to_excel(writer)

In [14]:
def save_data(index):
    '''save the index data to existing excel files'''
    
    tri_pe = get_data(index)
    
    wb = load_workbook(index+'.xlsx')
    ws = wb.active
    for i, r in tri_pe.iterrows():
        ws.append([i, r['Total Returns Index'], r['P/E'], r['P/B'], r['Div Yield']])
    wb.save(index+'.xlsx')

In [15]:
for i in indexes:
    save_data(i)

display last two daya of data from excel file

In [16]:
for i in indexes:
    print(i)
    display(pd.read_excel(i+'.xlsx').tail(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
3018,2023-03-06,25777.60,20.87,4.15,1.41
3019,2023-03-08,25840.12,20.92,4.16,1.40


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
3018,2023-03-06,23329.62,21.60,4.20,1.46
3019,2023-03-08,23392.36,21.66,4.22,1.46


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
3019,2023-03-06,5208.59,20.45,3.52,1.97
3020,2023-03-08,5217.83,20.49,3.53,1.96


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
3018,2023-03-06,11975.77,16.32,2.90,1.50
3019,2023-03-08,12001.25,16.36,2.91,1.49


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
3019,2023-03-06,11476.38,18.24,2.96,1.28
3020,2023-03-08,11507.23,18.29,2.97,1.28


To check whether same date repeats

In [17]:
for i in indexes:
    df=pd.read_excel(i+'.xlsx', index_col=0)
    print(i)
    display(df[df.index.duplicated(keep=False)])
    print(f"total duplicated indexes {df.index.duplicated(keep=False).sum()}\n")

nifty 50


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty 200


...

total duplicated indexes 2

nifty smallcap 100


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,


total duplicated indexes 0

nifty smallcap 250


,Total Returns Index,P/E,P/B,Div Yield
Date,,,,
2016-04-01,4742.59,65.64,1.54,1.14
2016-04-01,4742.59,86.86,1.35,1.20


total duplicated indexes 2



Starting date for which P/E values are available

In [18]:
for i in indexes:
    print(i)
    df = pd.read_excel(i+'.xlsx')
    display(df[~df['P/E'].isnull()].head(2))

nifty 50


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,7727.31,24.57,3.88,1.01
1,2011-01-04,7713.22,24.53,3.87,1.01


nifty 100


,Date,Total Returns Index,P/E,P/B,Div Yield
0,2011-01-03,6924.52,23.15,3.76,1.01
1,2011-01-04,6908.94,23.10,3.75,1.02


nifty 200


...

,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,3089.9,24.93,1.20,1.29
1302,2016-04-01,3089.9,27.61,0.99,1.52


nifty smallcap 100


,Date,Total Returns Index,P/E,P/B,Div Yield
1204,2015-11-09,6227.22,22.55,1.02,1.45
1205,2015-11-10,6144.27,22.5,1.00,1.47


nifty smallcap 250


,Date,Total Returns Index,P/E,P/B,Div Yield
1301,2016-04-01,4742.59,65.64,1.54,1.14
1302,2016-04-01,4742.59,86.86,1.35,1.20


In [19]:
nsepy.get_total_returns_index_history('nifty midcap 150', dt.date(2016,4,1), dt.date(2016,4,2))

,Total Returns Index
Date,
2016-04-01,4956.96


In [18]:
nsepy.get_index_pe_history('nifty midcap 150', dt.date(2016,4,1), dt.date(2016,4,2))

,P/E,P/B,Div Yield
Date,,,
2016-04-01,27.45,2.25,1.26
2016-04-01,27.76,2.21,1.20


Examples

In [6]:
sbi = nsepy.get_history('sbin', start=dt.date(2020,8,1), end=dt.date(2020,8,7))
sbi

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2020-08-03,SBIN,EQ,191.45,192.00,194.60,190.05,192.30,192.25,192.68,72457670,1.396079e+15,308823,9698233,0.1338
2020-08-04,SBIN,EQ,192.25,193.35,193.80,190.50,191.95,191.60,191.84,43867110,8.415652e+14,176987,6362150,0.1450
2020-08-05,SBIN,EQ,191.60,192.75,196.85,191.00,191.80,191.45,193.91,68888954,1.335821e+15,271605,9480012,0.1376
2020-08-06,SBIN,EQ,191.45,192.30,194.50,190.25,191.15,190.95,192.34,59740299,1.149069e+15,239032,7633674,0.1278
2020-08-07,SBIN,EQ,190.95,191.45,192.10,189.55,190.85,190.65,190.90,44822757,8.556666e+14,186761,9053179,0.2020


In [9]:
sbi.loc[dt.datetime.strptime('2020-08-03', '%Y-%m-%d').date()]['Open']

192.0

In [17]:
sbi.iloc[[1],[2]]

,Prev Close
Date,
2020-08-04,192.25


In [23]:
sbi.iat[1,2]

192.25